In [ ]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import pyautogui
import pyperclip
from pathlib import Path
from datetime import datetime
import warnings

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
from tkcalendar import DateEntry

def texto_cms_conf(nome,data_marcada):
    texto = f"""Olá {nome}, Falamos do Centro Médico Schorr de Alergia e Dermatologia para confirmar o seu agendamento do dia {data_marcada}.

*Avenida Nossa Senhora de Copacabana, 500 sala 711-712*

Entre as ruas Hilário de Gouveia e Paula Freitas

Metrô Siqueira Campos"""
    return texto

def texto_gsc_conf(nome,data_marcada):
    texto = f"""Olá {nome}, Falamos da Clínica Gerson Schorr de Alergia e Dermatologia para confirmar o seu agendamento do dia {data_marcada}.

*Av. Rio Branco, 257 sala 710-714*

Esquina com a Rua Santa Luzia

Metrô - Cinelâdia, saída E."""
    return texto

def texto_cms_repesc(nome,data_marcada):
    texto = f"""Olá {nome}, falamos do Centro Médico Schorr de Alergia e Dermatologia.

O sr(a) havia agendado uma consulta conosco dia {data_marcada} e não pode comparecer.

Gostaríamos de saber qual seria a disponibilidade do sr(a) para estar remarcando a sua consulta?

Aguardamos o seu retorno,obrigada."""
    return texto

def texto_gsc_repesc(nome,data_marcada):
    texto = f"""Olá {nome}, falamos da Clínica Gerson Schorr de Alergia e Dermatologia.

O sr(a) havia agendado uma consulta conosco dia {data_marcada} e não pode comparecer.

Gostaríamos de saber qual seria a disponibilidade do sr(a) para estar remarcando a sua consulta?

Aguardamos o seu retorno,obrigada."""
    return texto

# Formatar colunas de celular
def format_celular(nome_df, nome_coluna):
    nome_df[nome_coluna] = nome_df[nome_coluna].astype(str)

    nome_df[nome_coluna] = nome_df[nome_coluna].str.replace('-', '',regex=True)
    nome_df[nome_coluna] = nome_df[nome_coluna].str.replace('(', '',regex=True)
    nome_df[nome_coluna] = nome_df[nome_coluna].str.replace(')', '',regex=True)
    nome_df[nome_coluna] = nome_df[nome_coluna].str.replace(' ', '',regex=True)

    nome_df[nome_coluna] = nome_df[nome_coluna].astype(str)

    for i, celular in enumerate(nome_df[nome_coluna]):
        if len(celular) != 11:
            nome_df.loc[i, nome_coluna] = ' '

    for i, celular in enumerate(nome_df[nome_coluna]):
        if celular[0] == '0':
            nome_df.loc[i, nome_coluna] = ' '
                        
# Juntar Colunas de celular
def junta_coluna(nome_df, coluna_vazia, coluna_celular):
    for i, celular in enumerate(nome_df[coluna_vazia]):
        if celular == ' ':
            nome_df.loc[i, coluna_vazia] = nome_df.loc[i, coluna_celular]
            
def mandar_msg(tipo, nome, celular, data_marcada, empresa_escolhida):
    time.sleep(3)
    navegador.get('https://chat.sonax.net.br/#/app/chat')
    time.sleep(3)

    try:
        # Adicionar Contato
        navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
        navegador.find_element(By.XPATH,'//*[@id="adicionar-contato"]').click()
        navegador.find_element(By.XPATH,'//*[@id="MainEditContact"]/div[2]/div[1]/div/div/input').send_keys(nome)
        celular55 = '55' + celular             
        for i, carac_cel in enumerate(celular):                 
            navegador.find_element(By.XPATH,'//*[@id="phone"]').send_keys(celular[i])

        # Salvar Contato
        #navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
        navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/app-contact-form-chat/div/button[2]').click()
        time.sleep(3)

        # Se o paciente não tiver contato salvo no celular da clinica
        try:
        # Clicar na seta de mandar mensagem
            navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
            time.sleep(3)
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
            time.sleep(1)
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
            navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
            navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
            if tipo == 'conf':
                if empresa_escolhida == 'GSC':
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_gsc_conf(nome,data_marcada))
                else:
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_cms_conf(nome,data_marcada))
            else:
                if empresa_escolhida == 'GSC':
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_gsc_repesc(nome,data_marcada))
                else:
                    navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_cms_repesc(nome,data_marcada))
            navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

        # Se o paciente já tiver contato salvo no celular da clinica
        except:
            time.sleep(2)
            navegador.get('https://chat.sonax.net.br/#/app/chat')
            time.sleep(3)
            navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[1]/ul/li[2]/a').click()
            time.sleep(2)
            navegador.find_element(By.XPATH, '/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[1]/app-portlet/div/div/div/div[2]/div[1]/div/input').send_keys(celular)
            time.sleep(5)

            # Pegar lista de contatos com nome parecido
            lista_contatos_comparar = navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')

            tamanho_lista = len(lista_contatos_comparar)

            for x in range(tamanho_lista):
                if (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular55) or (navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].text == celular): # Atenção para Dtype do celular, precisa ser string
                    navegador.find_elements(By.CLASS_NAME, 'kt-widget__desc')[x].click()
            time.sleep(5)

            # Clicar na seta de mandar mensagem
            try: 
                navegador.find_element(By.XPATH,'/html/body/app-root/app-layout/ng-sidebar-container/div/div/app-chat/app-content/main/div/div/div[2]/app-portlet/div/div/app-contact/div[3]/div/table/tbody/tr/td[4]/button').click()
                time.sleep(3)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').click()
                time.sleep(1)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ARROW_DOWN)
                navegador.find_element(By.XPATH,'/html/body/ngb-modal-window/div/div/div[2]/div/div[3]/div/select').send_keys(Keys.ENTER)
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').clear()
                if tipo == 'conf':
                    if empresa_escolhida == 'GSC':
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_gsc_conf(nome,data_marcada))
                    else:
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_cms_conf(nome,data_marcada))
                else:
                    if empresa_escolhida == 'GSC':
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_gsc_repesc(nome,data_marcada))
                    else:
                        navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[2]/div/div[4]/div/textarea').send_keys(texto_cms_repesc(nome,data_marcada))
                navegador.find_element(By.XPATH, '/html/body/ngb-modal-window/div/div/div[3]/button[2]').click()

            # Caso já tenha uma conversa aberta com o paciente
            except:
                navegador.get('https://chat.sonax.net.br/#/app/chat')
                time.sleep(2)

    except: # Caso aconteça algum erro no salvamento do contato
        navegador.get('https://chat.sonax.net.br/#/app/chat')
        time.sleep(2)

            
def repescar(empresa_escolhida):
    global navegador
    
    pyautogui.alert(f"""Para pegar o relatório de repescagem {empresa_escolhida} no ASA:
    
    1- Agenda 
    2- Relatórios - Relatórios operacionais 
    3- Pacientes Faltosos da agenda médica 
    4- Selecionar "repesc gsc/cms hoje"
    5- Salvar como xlsx (excel)""")
        
    escolher_arquivo = askopenfilename(title=f'Favor, importar Repescagem {empresa_escolhida} em excel')
    repesc = pd.read_excel(escolher_arquivo) 
    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[1]/div/div[1]/input').send_keys(login)
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[1]/div/div[2]/input').send_keys(senha)
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[1]/div/button').click()
    time.sleep(3)
    
    # Tratando Planilha 
    
    repesc = repesc[['Unnamed: 1','Unnamed: 3','Unnamed: 4','Unnamed: 9']]
    repesc.columns = ['nome','data 2','data','celular']
    repesc = repesc.dropna(thresh=3)
    repesc = repesc[repesc['celular'] != 'Telefone']
    repesc = repesc.reset_index(drop=True)
    repesc['data'] = repesc['data'].fillna('x')

    # Unindo colunas de data
    lista_novas_datas = []

    for i, linha_data in enumerate(repesc['data']):
        if linha_data == 'x':
            linha_data = repesc.loc[i, 'data 2']
        lista_novas_datas.append(linha_data)

    repesc['data'] = lista_novas_datas

    repesc = repesc[['nome','data','celular']]

    # Mudando formatação da data
    lista_format_datas = []

    for data in repesc['data']:
        ano = data[6:10]
        mes = data[3:5]
        dia = data[0:2]
        hora = data[11:16]
        data = dia + '/' + mes + '/' + ano + ' ás ' + hora
        lista_format_datas.append(data)

    repesc['data'] = lista_format_datas

    # Tratando coluna dos celulares
    format_celular(repesc, 'celular')

    repesc = repesc.reset_index(drop=True)
    repesc['celular'] = repesc['celular'].astype(str)

    # Tratando coluna de nome dos pacientes
    lista_novos_nomes = []

    for i, nome_paciente in enumerate(repesc['nome']):
        nome_paciente = nome_paciente.title()
        lista_novos_nomes.append(nome_paciente)

    repesc['nome'] = lista_novos_nomes
    
    repesc['celular'] = repesc['celular'].astype(str)
    
    # -------------------------------------------------------------------------
    
    for i, celular in enumerate(repesc['celular']):
        nome = repesc.loc[i, 'nome']
        data_marcada = repesc.loc[i, 'data']
        print('Paciente {}'.format(nome))
        
        mandar_msg('repesc', nome, celular, data_marcada, empresa_escolhida)

    pyautogui.alert(f'Repescagem {empresa_escolhida} Finalizada!')
    navegador.quit()
    
def confirmar(empresa_escolhida):
    global navegador
    
    pyautogui.alert(f"""Para pegar o relatório de Confirmação {empresa_escolhida} no ASA:
    
    1- Agenda 
    2- Operações - Email/SMS 
    3- Filtrar Empresa para Centro Médico 
    4- Filtrar data do dia seguinte nos dois campos "atendido entre"
    5- Salvar como xlsx (excel)""")
    
    escolher_arquivo = askopenfilename(title=f'Favor, importar Confirmação {empresa_escolhida} em excel')
    confirmacao_bruto = pd.read_excel(escolher_arquivo) 

    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    navegador.get(r'https://chat.sonax.net.br/')

    navegador.maximize_window()

    time.sleep(3)
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[1]/div/div[1]/input').send_keys(login)
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[1]/div/div[2]/input').send_keys(senha)
    navegador.find_element(By.XPATH,'/html/body/app-root/app-login/div/div[1]/div/button').click()
    time.sleep(3)

        # Tratando Planilha Bruta --------------------------------------------------------------------------

    # Pegar Linha [2] da planilha
    novo_header = confirmacao_bruto.iloc[2]

    # Transformar essa linha [2] em lista
    novo_header = list(novo_header)

    # colocar essa lista como cabeçalho
    confirmacao_bruto.columns = novo_header

    # escolher colunas uteis da planilha com base no NOME delas
    confirmacao_bruto = confirmacao_bruto[['Código','Paciente','Data Marcada','Telefone 1','Telefone 2','Telefone 3']]
    confirmacao_bruto.columns = ['cod paciente', 'nome','data marcada', 'celular', 'celular 2', 'celular 3']

    confirmacao_bruto = confirmacao_bruto.dropna(subset=['nome']) 
    confirmacao_bruto = confirmacao_bruto[confirmacao_bruto['nome'] != 'Paciente']
    confirmacao_bruto = confirmacao_bruto.reset_index(drop=True)

    format_celular(confirmacao_bruto, 'celular')
    format_celular(confirmacao_bruto, 'celular 2')
    format_celular(confirmacao_bruto, 'celular 3')

    junta_coluna(confirmacao_bruto, 'celular', 'celular 2')
    confirmacao_bruto = confirmacao_bruto[['cod paciente', 'nome','data marcada', 'celular', 'celular 3']]

    junta_coluna(confirmacao_bruto, 'celular', 'celular 3')
    confirmacao_bruto = confirmacao_bruto[['cod paciente', 'nome', 'data marcada', 'celular']]    

    for i, celular in enumerate(confirmacao_bruto['celular']):        
        if celular == ' ':  
            confirmacao_bruto = confirmacao_bruto.drop(i)

    confirmacao_bruto = confirmacao_bruto.reset_index(drop=True)

    # Editando texto da data
    lista_novas_datas = []

    for data in confirmacao_bruto['data marcada']:
        ano = data[6:10]
        mes = data[3:5]
        dia = data[0:2]
        hora = data[11:16]
        data = dia + '/' + mes + '/' + ano + ' ás ' + hora
        lista_novas_datas.append(data)

    confirmacao_bruto['data marcada'] = lista_novas_datas

    confirmacao_bruto['celular'] = confirmacao_bruto['celular'].astype(str)

    # -----------------------------------------------------------------------------------------------------

    for i, celular in enumerate(confirmacao_bruto['celular']):
        nome = confirmacao_bruto.loc[i, 'nome']
        data_marcada = confirmacao_bruto.loc[i, 'data marcada']
        print('Paciente {}'.format(nome))

        mandar_msg('conf', nome, celular, data_marcada, empresa_escolhida)

    pyautogui.alert(f'Confirmação {empresa_escolhida} Finalizada!')
    navegador.quit()
      
janela = tk.Tk()

janela.wm_iconbitmap('wpp_azul.ico')

janela.title('Automação Sonax Whatsapp')

janela.geometry("%dx%d+%d+%d" % (370,120,500,300))

botao_gsc_con = tk.Button(text='CONFIRMAR GSC', bg='mediumorchid', fg='white', command=lambda: confirmar('GSC'), height=2, width=22)
botao_gsc_con.grid(row=0, column=0, padx=10, pady=10)

botao_cms_con = tk.Button(text='CONFIRMAR CMS', bg='skyblue', fg='black',command=lambda: confirmar('CMS'), height=2, width=22)
botao_cms_con.grid(row=0, column=1, padx=10, pady=10)

botao_gsc_rep = tk.Button(text='REPESCAR GSC', bg='mediumorchid', fg='white', command=lambda: repescar('GSC'), height=2, width=22)
botao_gsc_rep.grid(row=1, column=0, padx=2, pady=2)

botao_cms_rep = tk.Button(text='REPESCAR CMS', bg='skyblue', fg='black',command=lambda: repescar('CMS'), height=2, width=22)
botao_cms_rep.grid(row=1, column=1, padx=2, pady=2)

janela.mainloop()



Paciente Julia de Andrade Lima
Paciente Claudia Manoelina Costa Silva
Paciente Marcos da Conceicao Pinto da Silva
Paciente Alexandre Ribeiro Sucupira de Melo 
Paciente Aline da Gama Costa Nascimento
Paciente Viviane Rodrigues Valenca Silva
Paciente Priscila da Silva
Paciente Rodrigo Santos da Silva
Paciente Gabryelle Beatriz Passos dos Santos
Paciente Lucila Lima da Silva
Paciente Andreia Candido dos Anjos
Paciente Bianca Soares Oliveira
Paciente Mauricio Gotlib Costa
Paciente Jurema Ferreira da Silva
Paciente Selma do Nascimento
Paciente Alexandre da Franca Macedo
Paciente Joao Miguel Ferreira Sodre 
Paciente Vivian de Almeida Matos 
Paciente Agata Silveira Santos
Paciente Marcia de Almeida Pacheco 
Paciente REGINA CELIA DE SOUZA
Paciente NEY CAMARA DE CASTRO
Paciente Ceciliana Lima dos Santos
Paciente Artur Senna de Souza e Silva
Paciente Simone Santos Lima
Paciente Augusto Sergio da Silva
Paciente Marylene da Silva
Paciente Durcelina Soares de Lira 
Paciente Maira Guazzi
Paciente F